In [1]:
# import libraries
import urllib.request
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd 
from selenium.common.exceptions import NoSuchElementException

In [2]:
driver = webdriver.Chrome()

In [3]:
url = 'https://www.wildlifecrossing.net/california/observations/latest?q=observations/latest&page=0'

In [54]:
base_url = 'https://www.wildlifecrossing.net'

In [5]:
driver.get(url)

In [6]:
test = driver.find_elements_by_class_name("views-field views-field-title")

In [9]:
test = driver.find_elements_by_xpath("/html/body/div/div/div/section/div[2]/div/div/div/div/table/tbody/tr[1]/td[1]")

In [10]:
type(test)

list

In [11]:
len(test)

1

In [12]:
test

[<selenium.webdriver.remote.webelement.WebElement (session="bbff4566b2d1b812bfb7516866480f44", element="c967b7db-bcc3-43dc-b8c7-8eb732a719c4")>]

# Beautiful Souping

In [4]:
url = 'https://www.wildlifecrossing.net/california/observations/latest?q=observations/latest&page=0'
source = urllib.request.urlopen(url).read()
soup = BeautifulSoup(source,'lxml')

In [8]:
print(soup.title)

<title>Latest Observations | CROS</title>


In [9]:
print(soup.title.name)

title


In [10]:
print(soup.title.string)

Latest Observations | CROS


In [11]:
print(soup.title.parent.name)

head


In [12]:
print(soup.p)

<p>Developed by the <a href="https://roadecology.ucdavis.edu">Road Ecology Center</a>, <a href="https://www.ucdavis.edu">UC Davis</a></p>


<td headers="view-title-table-column" class="views-field views-field-title"><a href="/california/observations/61351" hreflang="en">Observation ID: 61351</a>        </td>

In [14]:
observation = soup.tr

In [15]:
print(observation)

<tr>
<th class="views-field views-field-title" id="view-title-table-column" scope="col">ID</th>
<th class="views-field views-field-field-date-observation" id="view-field-date-observation-table-column" scope="col">Date</th>
<th class="views-field views-field-field-taxon" id="view-field-taxon-table-column" scope="col">Animal</th>
</tr>


In [20]:
type(observation)

bs4.element.Tag

In [17]:
for paragraph in soup.find_all('p'):
    print(paragraph.string)
    print(str(paragraph.text))

None
Developed by the Road Ecology Center, UC Davis


In [21]:
for url in soup.find_all('a'):
    print(url.get('href'))

#main-content
/california/
/california/
/california/about
/california/map
https://roadecology.ucdavis.edu/hotspots/map
/california/user/login
/california/contact
None
/california/observations/61351
/california/observations/61350
/california/observations/61337
/california/observations/61347
/california/observations/61349
/california/observations/61332
/california/observations/61331
/california/observations/61345
/california/observations/61343
/california/observations/61341
/california/observations/61339
/california/observations/61326
/california/observations/61325
/california/observations/61324
/california/observations/61323
/california/observations/61320
/california/observations/61315
/california/observations/61334
/california/observations/61322
/california/observations/61313
/california/observations/61317
/california/observations/61314
/california/observations/61310
/california/observations/61312
/california/observations/61311
/california/observations/61355
/california/observations/61

## Navigation

In [22]:
nav = soup.nav

In [23]:
for url in nav.find_all('a'):
    print(url.get('href'))

/california/about
/california/map
https://roadecology.ucdavis.edu/hotspots/map


In [24]:
body = soup.body
for url in body.find_all('a'):
    print(url.get('href'))

#main-content
/california/
/california/
/california/about
/california/map
https://roadecology.ucdavis.edu/hotspots/map
/california/user/login
/california/contact
None
/california/observations/61351
/california/observations/61350
/california/observations/61337
/california/observations/61347
/california/observations/61349
/california/observations/61332
/california/observations/61331
/california/observations/61345
/california/observations/61343
/california/observations/61341
/california/observations/61339
/california/observations/61326
/california/observations/61325
/california/observations/61324
/california/observations/61323
/california/observations/61320
/california/observations/61315
/california/observations/61334
/california/observations/61322
/california/observations/61313
/california/observations/61317
/california/observations/61314
/california/observations/61310
/california/observations/61312
/california/observations/61311
/california/observations/61355
/california/observations/61

In [25]:
for div in soup.find_all('div', class_='body'):
    print(div.text)

In [27]:
table = soup.table

In [28]:
type(table)

bs4.element.Tag

In [140]:
%%time

page_num = 0

baseurl = 'https://www.wildlifecrossing.net'
url = '/california/observations/latest?q=observations/latest&page={}'.format(page_num)
source = urllib.request.urlopen(baseurl+url).read()
soup = BeautifulSoup(source,'lxml')

table = soup.find('table')
table_rows = table.find_all('tr')

data = {}
for tr in table_rows[1:]:
    suburl = base_url+tr.find('a').get('href')
    obsID = int(suburl.split('/')[-1])
    td = tr.find_all('td')
    _, time, critter = [i.text for i in td]
    subsource = urllib.request.urlopen(suburl).read()
    subsoup = BeautifulSoup(subsource,'lxml')
    meta = subsoup.find('div', {'class': 'geolocation-location js-hide'}).find_all('meta')
    data[obsID] = {
        'time': pd.to_datetime(time.strip()),
        'animal': critter.strip(),
        'url': suburl
    }
    for i in meta:
        data[obsID][i['property']] = float(i['content'])

CPU times: user 1.57 s, sys: 51.2 ms, total: 1.62 s
Wall time: 16.1 s


In [141]:
df = pd.DataFrame(data).T
df.to_json('data/CA_wildlife/{}.json'.format(str(0).rjust(4,'0')))

In [153]:
def scrape_ca_page(page_num):
    baseurl = 'https://www.wildlifecrossing.net'
    url = '/california/observations/latest?q=observations/latest&page={}'.format(page_num)
    source = urllib.request.urlopen(baseurl+url).read()
    soup = BeautifulSoup(source,'lxml')

    table = soup.find('table')
    table_rows = table.find_all('tr')

    data = {}
    for tr in table_rows[1:]:
        suburl = base_url+tr.find('a').get('href')
        obsID = int(suburl.split('/')[-1])
        td = tr.find_all('td')
        _, time, critter = [i.text for i in td]
        subsource = urllib.request.urlopen(suburl).read()
        subsoup = BeautifulSoup(subsource,'lxml')
        
        data[obsID] = {
            'time': pd.to_datetime(time.strip()),
            'animal': critter.strip(),
            'url': suburl
        }
        try:
            meta = subsoup.find('div', {'class': 'geolocation-location js-hide'}).find_all('meta')
            for i in meta:
                data[obsID][i['property']] = float(i['content'])
        except:
            data[obsID]['latitude'] = None
            data[obsID]['longitude'] = None
            
    df = pd.DataFrame(data).T
    df.to_json('data/CA_wildlife/{}.json'.format(str(page_num).rjust(4,'0')))

In [146]:
%%time
scrape_ca_page(1)

CPU times: user 1.61 s, sys: 52.4 ms, total: 1.67 s
Wall time: 54.4 s


In [154]:
%%time
for page_num in range(2, 12):
    print('{}, '.format(page_num), end='')
    scrape_ca_page(page_num)

2, 3, 4, 5, 6, 7, 8, 9, 10, 11, CPU times: user 16.6 s, sys: 343 ms, total: 17 s
Wall time: 8min 56s


In [129]:
df.head()

,time,animal,url,latitude,longitude
61351,2019-09-16 10:43:00,California Ground Squirrel,https://www.wildlifecrossing.net/california/ob...,34.3269,-119.108
61350,2019-09-16 10:41:00,Eastern Fox Squirrel,https://www.wildlifecrossing.net/california/ob...,34.3269,-119.108
61337,2019-09-15 11:55:00,Mule (or Black tailed) Deer,https://www.wildlifecrossing.net/california/ob...,34.3269,-119.108
61347,2019-09-15 09:32:00,Western Fence Lizard,https://www.wildlifecrossing.net/california/ob...,34.3269,-119.108
61349,2019-09-15 08:38:00,Gopher Snake,https://www.wildlifecrossing.net/california/ob...,34.3269,-119.108


20400

In [147]:
*1200/3600

SyntaxError: unexpected EOF while parsing (<ipython-input-147-750558c2cde7>, line 1)

In [132]:
str(1).rjust(4,'0')

'0001'

# Scraping observations

In [82]:
test_url = df.loc[61351, 'url']

In [83]:
source = urllib.request.urlopen(test_url).read()
soup = BeautifulSoup(source,'lxml')

In [87]:
test = soup.find_all('meta')

In [88]:
test

[<meta charset="utf-8"/>,
 <meta content="Drupal 8 (https://www.drupal.org)" name="Generator"/>,
 <meta content="width" name="MobileOptimized"/>,
 <meta content="true" name="HandheldFriendly"/>,
 <meta content="width=device-width, initial-scale=1.0" name="viewport"/>,
 <meta content="34.326941420035" property="latitude"/>,
 <meta content="-119.10751861418" property="longitude"/>]

In [91]:
soup.body.children

In [114]:
meta = soup.find('div', {'class': 'geolocation-location js-hide'}).find_all('meta')

In [123]:
meta[0].attrs

{'property': 'latitude', 'content': '34.326941420035'}

In [118]:
for i in meta:
    print("{}: {}".format(i['property'], float(i['content'])))

latitude: 34.326941420035
longitude: -119.10751861418


In [119]:
["{}: {}".format(i['property'], float(i['content'])) for i in meta]

['latitude: 34.326941420035', 'longitude: -119.10751861418']

In [102]:
type(div.find_all('meta')[0])

bs4.element.Tag

In [103]:
div.find_all('meta')[0].attrs

{'property': 'latitude', 'content': '34.326941420035'}

In [104]:
div.find_all('meta')[1].attrs

{'property': 'longitude', 'content': '-119.10751861418'}

In [109]:
div.find_all('meta')[1]['property']

'longitude'

In [111]:
float(div.find_all('meta')[1]['content'])

-119.10751861418

In [108]:
300/60

5.0